# This code is for ModelC
build source model based on source countries data(CCPM and control measures) and predict CCPM for target countries

In [1]:
# Part 1 - Data Preprocessing
import tensorflow as tf
# Importing the libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import os
from tqdm import tqdm
import datetime 


# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model,clone_model
from keras.layers import Input, Embedding, LSTM, Dense,  Lambda

from keras.backend import slice
from keras.constraints import max_norm
import warnings
warnings.simplefilter('ignore')
os.environ["CUDA_VISIBLE_DEVICES"]='-1'

##　locate the directory storing the data 
os.chdir(os.getcwd()+'/data')

/data/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [3]:
def create_sequences_x(data, seq_length):
    xs = []
    for i in range(len(data)-seq_length+1):
        x = data[i:(i+seq_length)]
        xs.append(x)
    return np.array(xs)


def create_sequences_y(data, seq_length):
    ys = []
    for i in range(seq_length, len(data)):
        y = data[i]
        ys.append(y)
    return np.array(ys)

def MAPE(y, y_pred):
    mape = sum(abs(y-y_pred)/y)/len(y)
    print('MAPE: ', mape)
    return mape

# 构造source序列

In [4]:
from os import listdir
from os.path import isfile, join
from re import sub

mypath = 'source/'
source_countries = ['Austria','China (except Hubei)','Croatia','Germany','Hubei','Italy','Japan',
            'Lebanon','Monaco','Norway','Oman','United Arab Emirates']

(['Croatia',
  'Norway',
  'Lebanon',
  'Hubei',
  'Austria',
  'China (except Hubei)',
  'Oman',
  'United Arab Emirates',
  'Japan',
  'Germany',
  'Italy',
  'Monaco'],
 12)

In [5]:
pred_length = 7
seq_length = 7
x_seq0 = []
y_seq0 = []

for sc in source_countries:
    print(sc)
    # data preprocessing
    df = pd.read_excel('source/'+sc+'.xlsx', index_col=0)

    df_new_1day = df[['confirmed cases per million']].diff(periods=1)
    df_new_1day.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)
    df_new_1day['control'] = df.control.values

    df_new_7days = df[['confirmed cases per million']].diff(periods=pred_length)
    df_new_7days.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)

    df_new_7days['cum cases'] = 0
    df_new_7days['cum cases'][pred_length:] = df['confirmed cases per million'].values[0:len(df_new_7days)-pred_length]  
    
    ## scale data
    scaler_x = MinMaxScaler() #scale data into 0-1
    scaler_y = MinMaxScaler() 

    if len(x_seq0)==0:
        x_seq0 = create_sequences_x(df_new_1day.dropna().to_numpy(), seq_length)
        y_seq0 = create_sequences_y(df_new_7days.dropna().to_numpy(), seq_length)
        x_seq0 = x_seq0[0:len(y_seq0)]
    else:
        tx_seq0 = create_sequences_x(df_new_1day.dropna().to_numpy(), seq_length)
        ty_seq0 = create_sequences_y(df_new_7days.dropna().to_numpy(), seq_length)
        tx_seq0 = tx_seq0[0:len(ty_seq0)]
        
        x_seq0 = np.concatenate((x_seq0, tx_seq0),axis=0)
        y_seq0 = np.concatenate((y_seq0, ty_seq0),axis=0)

Croatia
Norway
Lebanon
Hubei
Austria
China (except Hubei)
Oman
United Arab Emirates
Japan
Germany
Italy
Monaco


# scale the sequence values

In [7]:
x_seq1 = np.reshape(x_seq0, newshape=(-1,1))

scaler_x = scaler_x.fit(x_seq1)
x = scaler_x.transform(x_seq1)
x = np.reshape(x, newshape=(x_seq0.shape))
y_seq1 = np.reshape(y_seq0[:,0:1], newshape=(-1,1))

scaler_y = scaler_y.fit(y_seq1)
y = scaler_y.transform(y_seq1)

y = np.reshape(y, newshape=(y_seq0[:,0:1].shape))

# 训练source模型

In [8]:
from random import seed
from random import sample

In [15]:
seed(123)
train_idx = sample(range(len(x)),300)
test_idx = set(range(len(x))).difference(train_idx)


X_train = x[train_idx].copy()
y_train = np.reshape(y[train_idx], (-1))

X_test = x[list(test_idx)].copy()
y_test = np.reshape(y[list(test_idx)], (-1))

# Building the model
main_input = Input(shape=(seq_length,2,), dtype='float32', name='main_input')  
lstm_out = LSTM(4)(main_input)   
main_output = Dense(units = 1)(lstm_out)
regressor = Model(inputs=main_input, outputs=main_output)


regressor.compile(optimizer='adam', loss='mse')
regressor.fit(X_train, y_train, epochs = 50, batch_size = 32, validation_data=(X_test,y_test))
regressor.save('../model/ModelC-control.pkl')

Train on 300 samples, validate on 593 samples
Epoch 1/50
300/300 [==============================] - 2s 6ms/step - loss: 0.0306 - val_loss: 0.0222
Epoch 2/50
300/300 [==============================] - 0s 392us/step - loss: 0.0254 - val_loss: 0.0196
Epoch 3/50
300/300 [==============================] - 0s 399us/step - loss: 0.0230 - val_loss: 0.0186
Epoch 4/50
300/300 [==============================] - 0s 376us/step - loss: 0.0221 - val_loss: 0.0184
Epoch 5/50
300/300 [==============================] - 0s 373us/step - loss: 0.0218 - val_loss: 0.0183
Epoch 6/50
300/300 [==============================] - 0s 366us/step - loss: 0.0215 - val_loss: 0.0179
Epoch 7/50
300/300 [==============================] - 0s 360us/step - loss: 0.0211 - val_loss: 0.0176
Epoch 8/50
300/300 [==============================] - 0s 369us/step - loss: 0.0208 - val_loss: 0.0173
Epoch 9/50
300/300 [==============================] - 0s 365us/step - loss: 0.0206 - val_loss: 0.0170
Epoch 10/50
300/300 [=================

141.9056389988901

# target countries - medium

In [1]:
from os import listdir
from os.path import isfile, join
from re import sub
target_countries = ['Albania','Algeria','Argentina','Armenia','Australia','Azerbaijan','Bangladesh','Belarus','Belgium','Bermuda',
             'Bolivia','Brazil','Bulgaria','Canada','Chile','Colombia','Costa Rica','Cuba','Czech Republic','Denmark',
             'El Salvador','Estonia','Finland','France','Ghana','Gibraltar','Greece','Honduras','Hungary','India',
             'Indonesia','Iran','Iraq','Ireland','Israel','Jamaica','Jordan','Liberia','Luxembourg','Malaysia','Mexico',
             'Morocco','Nepal','Netherlands','Nigeria','Pakistan','Paraguay','Peru','Philippines','Poland','Portugal',
             'Qatar','Republic of the Congo','Romania','Russia','Rwanda','Saudi Arabia','Senegal','Sierra Leone',
             'Singapore','Slovakia','Slovenia','South Africa','Sri Lanka','Switzerland','Thailand','Tunisia','Turkey',
             'Ukraine','United Kingdom','United States','Venezuela']

In [13]:
pred_model = load_model('../model/ModelC-control.pkl')
ret_test = pd.DataFrame(index=target_countries, columns=['MAPE'])
modelc_control = pd.DataFrame(index=['loop1', 'loop2', 'loop3', 'loop4','loop5'], columns=['MAPE(mean)','MAPE(std)','MAPE<0.1','MAPE<0.05'])

In [17]:
mypath = 'target/'
for l in range(5):
    for tar in target_countries:
        print(tar)
        # data preprocessing
        df = pd.read_excel(mypath+tar+'.xlsx', index_col=0)

        df_new_1day = df.diff(periods=1)
        df_new_1day.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)

        df_new_1day['control'] = df.control.values

        df_new_7days = df[['confirmed cases per million']].diff(periods=pred_length)
        df_new_7days.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)

        df_new_7days['cum cases'] = 0
        df_new_7days['cum cases'][pred_length:] = df['confirmed cases per million'].values[0:len(df_new_7days)-pred_length]

        x_seq0 = create_sequences_x(df_new_1day.dropna().to_numpy(), seq_length)
        y_seq0 = create_sequences_y(df_new_7days.dropna().to_numpy(), seq_length)
        x_seq0 = x_seq0[0:len(y_seq0)]

        x_seq1 = np.reshape(x_seq0, newshape=(-1,1))
        x = scaler_x.transform(x_seq1)

        x = np.reshape(x, newshape=(x_seq0.shape))

        y_seq1 = np.reshape(y_seq0[:,0:1], newshape=(-1,1))
        y = scaler_y.transform(y_seq1)

        y = np.reshape(y, newshape=(y_seq0[:,0:1].shape))

        test_idx = int(len(x)*0.8)
        X_train = x[0:test_idx, :, 0:2].copy()
        y_train = np.reshape(y[0:test_idx], (-1))

        X_test = x[test_idx:,:,0:2].copy()
        y_test = np.reshape(y[test_idx:], (-1))

        model =clone_model(pred_model)
        model.set_weights(pred_model.get_weights())
        for layer in model.layers[:-1]:
            layer.trainable=False
        for layer in model.layers[-1:]:
            layer.trainable=True
        model.compile(optimizer='adam', loss='mse')
        # Fitting the RNN to the Training set
        model.fit(X_train, y_train, epochs = 12, batch_size = 2, verbose=0)

        # Predicting daily cases
        predicted_cases = model.predict(X_test)
        predicted_cases = scaler_y.inverse_transform(predicted_cases)

        true_cases = np.reshape(scaler_y.inverse_transform(np.reshape(y_test,(-1,1))),(-1))+y_seq0[test_idx:,1]
        predicted_cases = np.reshape(predicted_cases,(-1))+y_seq0[test_idx:,1]


        mape = MAPE(true_cases, predicted_cases) 
        ret_test.loc[tar, 'MAPE'] = mape
    modelc_control.loc['loop'+str(l+1), 'MAPE(mean)'] = ret_test.MAPE.mean() 
    modelc_control.loc['loop'+str(l+1), 'MAPE(std)'] = ret_test.MAPE.std()
    modelc_control.loc['loop'+str(l+1), 'MAPE<0.1'] = ret_test[(ret_test.MAPE<0.1)].shape[0]
    modelc_control.loc['loop'+str(l+1), 'MAPE<0.05']=ret_test[(ret_test.MAPE<0.05)].shape[0]

Iraq
MAPE:  0.1010970806296633
Denmark
MAPE:  0.03745080750703928
Republic of the Congo
MAPE:  0.03700844144024166
Slovakia
MAPE:  0.04057251380286587
Azerbaijan
MAPE:  0.06491090043406075
Australia
MAPE:  0.04540069997822493
Bolivia
MAPE:  0.16201956809349902
Bangladesh
MAPE:  0.10171506357672477
Indonesia
MAPE:  0.04695581045680596
Estonia
MAPE:  0.03193089629341609
Rwanda
MAPE:  0.09133714841478137
Nigeria
MAPE:  0.06854949295051346
Bermuda
MAPE:  0.030609473925073495
Jordan
MAPE:  0.039542500962664644
Argentina
MAPE:  0.15328290571695996
Finland
MAPE:  0.047290475889908534
Paraguay
MAPE:  0.07748345325170151
Saudi Arabia
MAPE:  0.05200179923381
Turkey
MAPE:  0.039656551317122375
Ireland
MAPE:  0.04031761899627622
Iran
MAPE:  0.015912292409801902
Switzerland
MAPE:  0.03872265756354139
Colombia
MAPE:  0.08940600138766834
Albania
MAPE:  0.023755701435803786
Pakistan
MAPE:  0.06056629996624596
Belgium
MAPE:  0.03445783736177821
United States
MAPE:  0.016500321648057793
Israel
MAPE:  0.

In [16]:
modelc_control.to_csv('../result/modelC_control.csv')